In [1]:
import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

### Extraction

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 100 Documents


In [4]:
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

In [5]:
from hardware_utils import OmniNgramsHardware

hardware_ngrams = OmniNgramsHardware(n_max=3, split_tokens=None)

In [6]:
from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)

Loaded 1286 part numbers.


In [7]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [hardware_ngrams], [parts_matcher])

In [8]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 3min 19s, sys: 4.55 s, total: 3min 24s
Wall time: 3min 26s
Candidate Set (Hardware Candidates) contains 16969 Candidates


### Saving Candidates

In [9]:
session.add(candidates)
session.commit()

In [10]:
import os
os.system('cp snorkel.db snorkel.db\ candidates');

### Reloading Candidates

In [11]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ candidates snorkel.db');

from snorkel.models import Corpus
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()

In [12]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

Candidate Set (Hardware Candidates) contains 16969 Candidates


### Error Analysis

In [20]:
print len(candidates)
print candidates[0]

16969
Part(ImplicitSpan("MMBT3904-16", parent=44040, words=[0,0], position=[1]))


In [13]:
# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, 'stg_temp_min')

Preparing gold set...
Preparing candidates...
[========================================] 100%

Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 2196 
Entity-level Gold: 542
Intersection Candidates: 541
----------------------------------------
Overlap with Gold:  1.00



In [14]:
for c in list(fn):
    print c

('PJECS00521-1', 'MMBT3904')


In [15]:
print candidates[0].part.parent.document.name

MCCCS08984-1


In [16]:
from hardware_utils import expand_part_range

for p in expand_part_range("BC 856 A"): print p

BC856
BC856-16
BC856-25
BC856-40
BC856A
BC856B
BC856C


In [17]:
"BC 856 A" in parts_dict

True

In [ ]:
for c in candidates:
    if c.part.parent.document.name == "SIEMS01215-1":
        import pdb; pdb.set_trace()

In [ ]:
# for c in candidates:
#     if c.part.get_span() == 'BC858A':
#         import pdb; pdb.set_trace()

In [ ]:
# def get_doc(corpus, doc_name):
#     for doc in corpus.documents:
#         if doc.name.upper() == doc_name.upper():
#             return doc

In [ ]:
for c in candidates[100:200]:
    print c

In [ ]:
# doc = get_doc(corpus, 'Infineon-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-v01_01-en')
# for phrase in doc.phrases:
#     if 'BC857CW' in phrase.words:
#         import pdb; pdb.set_trace()